# 2.1 Members Data Cleaning

##### Description

Basic data visualization and data formatting for members.csv

##### Notebook Steps

1. Connect Spark
1. Input Data
1. Examine Data
1. Data Cleaning
1. Output Data

## 1. Connect Spark

In [1]:
%load_ext sparkmagic.magics

In [2]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

Added endpoint http://ec2-3-85-12-54.compute-1.amazonaws.com:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1609241788334_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
Cleaned up endpoint http://ec2-3-85-12-54.compute-1.amazonaws.com:8998/


## 2. Input Data

In [1]:
%%spark

df = spark.read.csv("s3://jolfr-capstone3/raw/members", header=True)

UsageError: Cell magic `%%spark` not found.


## 3. Examine Data

##### show()

In [4]:
%%spark
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  null|            11|              20110911|
|+tJonkh+O1CA796Fm...|   1|  0|  null|             7|              20110914|
|cV358ssn7a0f7jZOw...|   1|  0|  null|            11|              20110915|
|9bzDeJP6sQodK73K5...|   1|  0|  null|            11|              20110915|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|
|yLkV2gbZ4GLFwqTOX...|   4| 30|  male|             9|              20110916|
|jNCGK78YkTyId3H3w...|   1|  0|  null|             7|              20110916|
|WH5Jq4mgtfUFXh2yz...|   5| 34|  male|             9|              20110916|
|tKmbR4X5VXjHmxERr...|   5| 19|  male|             9|              20110917|
|I0yFvqMoNkM8ZNHb6...|  13| 63|  male|             9|              20110918|

##### count()

In [5]:
%%spark
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6769473

##### describe()

In [6]:
%%spark
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------------+------------------+-------+-----------------+----------------------+
|summary|                msno|             city|                bd| gender|   registered_via|registration_init_time|
+-------+--------------------+-----------------+------------------+-------+-----------------+----------------------+
|  count|             6769473|          6769473|           6769473|2339968|          6769473|               6769473|
|   mean|                null|3.847357689439045| 9.795794295951842|   null|5.253068739619761|  2.0145175906336136E7|
| stddev|                null| 5.47835906326049|17.925899719010864|   null|2.361398314806483|     23186.00761211712|
|    min|+++4vcS9aMH7KWdfh...|                1|               -10| female|               -1|              20040326|
|    max|zzzyOgMk9MljCerbC...|                9|               994|   male|                9|              20170429|
+-------+--------------------+-----------------+----------------

##### printSchema()

In [7]:
%%spark
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- msno: string (nullable = true)
 |-- city: string (nullable = true)
 |-- bd: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: string (nullable = true)
 |-- registration_init_time: string (nullable = true)

##### columns

In [8]:
%%spark
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['msno', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time']

##### head(5)

In [9]:
%%spark
df.head(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(msno=u'Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=', city=u'1', bd=u'0', gender=None, registered_via=u'11', registration_init_time=u'20110911'), Row(msno=u'+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=', city=u'1', bd=u'0', gender=None, registered_via=u'7', registration_init_time=u'20110914'), Row(msno=u'cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=', city=u'1', bd=u'0', gender=None, registered_via=u'11', registration_init_time=u'20110915'), Row(msno=u'9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=', city=u'1', bd=u'0', gender=None, registered_via=u'11', registration_init_time=u'20110915'), Row(msno=u'WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=', city=u'6', bd=u'32', gender=u'female', registered_via=u'9', registration_init_time=u'20110915')]

##### Null per Column

In [10]:
%%spark
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+---+-------+--------------+----------------------+
|msno|city| bd| gender|registered_via|registration_init_time|
+----+----+---+-------+--------------+----------------------+
|   0|   0|  0|4429505|             0|                     0|
+----+----+---+-------+--------------+----------------------+

##### Value Counts

In [11]:
%%spark
df.groupBy('city').count().orderBy('count').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|city| count|
+----+------+
|  19|  1199|
|  20|  4233|
|  16|  5092|
|   7| 11610|
|   3| 27282|
|  17| 27772|
|  21| 30837|
|  10| 32482|
|  18| 38039|
|   8| 45975|
|  11| 47489|
|   9| 47639|
|  12| 66843|
|  14| 89940|
|   6|135200|
|  15|190213|
|  22|210407|
|   4|246848|
|  13|320978|
|   5|385069|
+----+------+
only showing top 20 rows

In [12]:
%%spark
df.groupBy('bd').count().orderBy('count').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
|   bd|count|
+-----+-----+
| 1111|    1|
| 1954|    1|
|  164|    1|
| 1053|    1|
| 1821|    1|
|  826|    1|
|  334|    1|
|  462|    1|
|-6807|    1|
|  743|    1|
|  183|    1|
|  685|    1|
| 1062|    1|
| 1213|    1|
| 1058|    1|
|  935|    1|
|   -8|    1|
|  573|    1|
| -178|    1|
|  230|    1|
+-----+-----+
only showing top 20 rows

In [13]:
%%spark
df.groupBy('gender').count().orderBy('count').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+
|gender|  count|
+------+-------+
|female|1144613|
|  male|1195355|
|  null|4429505|
+------+-------+

In [14]:
%%spark
df.groupBy('registered_via').count().orderBy('count').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|registered_via|  count|
+--------------+-------+
|            -1|      1|
|            18|      5|
|            10|     10|
|             1|     43|
|            14|    615|
|            16|    888|
|            19|    974|
|             6|   1213|
|             2|   1452|
|            17|   1494|
|             5|   3115|
|             8|   3982|
|            13|   5455|
|            11|  25047|
|             7| 805895|
|             9|1482863|
|             3|1643208|
|             4|2793213|
+--------------+-------+

## 4. Data Cleaning

In [15]:
%%spark
from pyspark.sql import types
from pyspark.sql.functions import col, to_date, regexp_replace, when
from operator import add

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Columns

##### msno
The msno column corresponds to user ids for the dataset, so the column is renamed from msno to user_id.

In [16]:
%%spark
df = df.withColumnRenamed("msno","user_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### city
The city column is currently formatted as a column of strings. The actual data is represented as integers, so the column is cast to match.

In [17]:
%%spark
df = df.withColumn("city",col("city").cast(types.IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### bd
The bd column has a high number of outlier values, so the column will be dropped.

In [18]:
%%spark
df = df.drop(df.bd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### gender
The gender column has a high number of null values, so the column will be dropped.

In [19]:
%%spark
df = df.drop(df.gender)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### registered_via
As with the city column, the registered_via column is cast from string to integer.

In [20]:
%%spark
df = df.withColumn("registered_via",col("registered_via").cast(types.IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### registration_init_time
The registration_init_time column must be parsed and cast to a date object. The column is also renamed to registration_date.

In [21]:
%%spark
df= df.withColumn('registration_init_time',to_date(df.registration_init_time, 'yyyyMMdd'))
df = df.withColumnRenamed("registration_init_time", "registration_date")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 5. Data Output

##### Final Check

In [22]:
%%spark
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- city: integer (nullable = true)
 |-- registered_via: integer (nullable = true)
 |-- registration_date: date (nullable = true)

In [23]:
%%spark
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------------+-----------------+
|             user_id|city|registered_via|registration_date|
+--------------------+----+--------------+-----------------+
|Rb9UwLQTrxzBVwCB6...|   1|            11|       2011-09-11|
|+tJonkh+O1CA796Fm...|   1|             7|       2011-09-14|
|cV358ssn7a0f7jZOw...|   1|            11|       2011-09-15|
|9bzDeJP6sQodK73K5...|   1|            11|       2011-09-15|
|WFLY3s7z4EZsieHCt...|   6|             9|       2011-09-15|
|yLkV2gbZ4GLFwqTOX...|   4|             9|       2011-09-16|
|jNCGK78YkTyId3H3w...|   1|             7|       2011-09-16|
|WH5Jq4mgtfUFXh2yz...|   5|             9|       2011-09-16|
|tKmbR4X5VXjHmxERr...|   5|             9|       2011-09-17|
|I0yFvqMoNkM8ZNHb6...|  13|             9|       2011-09-18|
|OoDwiKZM+ZGr9P3fR...|   1|             7|       2011-09-18|
|dCvvBHlaOAqgkAcv3...|  22|             9|       2011-09-19|
|6bra2AiVV8SGlm7R6...|   4|             9|       2011-09-19|
|4De1jAxNRABoyRBDZ...|  

##### Output to File

In [25]:
%%spark

df.write.format("com.databricks.spark.csv").option("header", "true").mode('overwrite').save('s3://jolfr-capstone3/interim/members')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…